# Exploring Amazon Metadata Dataset

## Importing Libs

In [1]:
import pyspark.pandas as ps
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import pyspark.sql.functions as F
from pyspark.sql.types import StructType,StructField, StringType

StatementMeta(synsp03, 9, 2, Finished, Available)

'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.


In [16]:
spark.conf.get('spark.kryoserializer.buffer.max')

StatementMeta(synsp03, 8, 2, Finished, Available)

'512m'

## Loading dataframe

In [17]:
root = 'abfss://default@stdatalakeakita.dfs.core.windows.net/synapse/workspaces/syn-synfactoreddatathon01-dev'

df = spark.read\
.format('parquet')\
.load(f'{root}/silver/silver_amazon_metadata/**')\
.withColumn('secondary_category', F.col('category').getItem(1))\
.repartition(200)

df_pandas = df.pandas_api()

StatementMeta(synsp03, 9, 18, Finished, Available)

## Exploratory Data Analysis

In [18]:
print(df_pandas.shape)
print(df_pandas.columns)

StatementMeta(synsp03, 9, 19, Finished, Available)

(14741571, 13)
Index(['asin', 'also_buy', 'also_view', 'brand', 'category', 'main_category',
       'description', 'feature', 'image', 'price', 'rank', 'title',
       'secondary_category'],
      dtype='object')


In [19]:
df_pandas.head(20)

StatementMeta(synsp03, 9, 20, Finished, Available)

,asin,also_buy,also_view,brand,category,main_category,description,feature,image,price,rank,title,secondary_category
0,B014LRS176,[],"[B01LWTYVF6, B00JT3DDJO, B01HT0QFHA, B07DL2D56...",QUALIZZI,"[Home & Kitchen, Seasonal Dcor]",Home & Kitchen,FAIRY LIGHTS a PACK OF 3 Warm white mini LED l...,300 MINI LED Bulbs and Three AC/DC Power Adapt...,https://images-na.ssl-images-amazon.com/images...,$47.99,433688,QUALIZZI Fairy Lights Set - String Lights That...,Seasonal Dcor
1,B00LCSWV4C,[],[],Bula,"[Sports & Outdoors, Outdoor Recreation, Winter...",Sports & Outdoors,Sheep wool will never feel itchy against your ...,50% MERINO WOOL 50% ACRYLIC,,,1665627,Bula Women's Fiasco Beanie,Outdoor Recreation
2,B00IXYFF7S,[B00IXYJBAU],[],PTC,"[Home & Kitchen, Home Dcor, Home Dcor Accents,...",Home & Kitchen,This gorgeous 6.75 Inch Screaming Werewolf Myt...,Perfect gift for those that love Gargoyle,,,2309232,6.75 Inch Screaming Werewolf Mythological Crea...,Home Dcor
3,B00HYJJ4Y8,[],[],Monster Jam,[],,Off-Road Series,1:64,,$9.77,866407,Hot Wheels Off-Road Monster Jam 2014 New Look ...,None
4,B00B5ZWY7G,[],[],GeauxRobot,"[Electronics, Computers & Accessories, Compute...",Electronics,Compatible with Raspberry Pi V2.0 model. The c...,Compatible with Raspberry Pi V2.0,https://images-na.ssl-images-amazon.com/images...,,2923,GeauxRobot Clear Raspberry Pi Case Box Enclosure,Computers & Accessories
5,B0094KW3E8,[],"[B078HBZ9NH, B07CP1SCCJ, B01M6B9BHX, B07CJQ857...",,"[Clothing, Shoes & Jewelry, Men, Shoes, Athle...",Clothing,,<span class=\a-text-bold\>Package Dimensions:\...,https://images-na.ssl-images-amazon.com/images...,,6252740,"Brooks Men's Cascadia 8 Trail Running Shoes, C...",Shoes & Jewelry
6,B00JBD2CSK,"[B00N5WOP38, B00A6SP5EC, B00N5WP8UM, B00N5WPAY...",[],Mattel,"[Toys & Games, Toy Remote Control & Play Vehic...",Toys & Games,Favorite characters from Disney/Pixar Cars and...,Enchanting die-cast assortment gives kids the ...,https://images-na.ssl-images-amazon.com/images...,$9.97,457770,Mattel Disney/Pixar Cars Nigel Gearsley Diecas...,Toy Remote Control & Play Vehicles
7,B009K6TDQW,[],"[B00012NJIC, B00172XUWU, B0013OQJ66, B0043ZLTT...",Adiosa,[],,,,https://images-na.ssl-images-amazon.com/images...,$16.69,56513,Adiosa Calcium Citrate with Black Current for ...,None
8,B00G6UW8OO,[],[],Opl auto parts,"[Automotive, Exterior Accessories, Towing Prod...",Automotive,Designed to handle what drivers who push their...,Aluminum Universal 7 Row Transmission/Oil Cool...,https://images-na.ssl-images-amazon.com/images...,$22.00,2013242,OPL HPO014 Universal Aluminum Oil Cooler,Exterior Accessories
9,1592336019,"[1401942822, 1401951260, 1401947832, 161212433...","[1612124259, 1623155282, 1401951260, 161564150...",Meg Thompson,"[Books, Cookbooks, Food & Wine, Cooking by In...",Books,Meg Thompson is a naturopath,,,$9.96,641529,"Superfoods for Life, Cultured and Fermented Be...",Cookbooks


In [20]:
df_pandas.nunique(axis=0)

StatementMeta(synsp03, 9, 21, Finished, Available)

/opt/spark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


also_view              4326054
brand                  2307902
description            7090681
feature                3687210
rank                   6402836
title                 14127828
asin                  14741571
also_buy               3612263
category               1475085
main_category               30
image                  5486399
price                   236142
secondary_category         574
dtype: int64

In [26]:
print(df_pandas['main_category'].unique())
print(df_pandas['main_category'].value_counts())

StatementMeta(synsp03, 9, 27, Finished, Available)

0                  Alexa Skills
1                Home & Kitchen
2                  Pet Supplies
3                  Toys & Games
4                         Patio
5             Sports & Outdoors
6                   Electronics
7                   Video Games
8                      Clothing
9                   CDs & Vinyl
10                   Automotive
11                        Books
12      Industrial & Scientific
13    Cell Phones & Accessories
14       Magazine Subscriptions
15                         Arts
16                     Software
17     Tools & Home Improvement
18                  Movies & TV
19                 Kindle Store
20       Grocery & Gourmet Food
21              Office Products
22                   Appliances
23      Collectibles & Fine Art
24     Home & Business Services
25          Musical Instruments
26                             
27                   Gift Cards
28                     Handmade
29                Digital Music
Name: main_category, dtype: object
Cloth

## Let's focus on Automotive category

In [32]:
display(df.where('main_category = "Automotive"').groupBy('secondary_category').count().orderBy(F.col('count').desc()))

StatementMeta(synsp03, 9, 33, Finished, Available)

SynapseWidget(Synapse.DataFrame, fdd62ac5-99a4-4fdd-a099-84b1f8acec8e)

### Cleaning secondary category

In [34]:
df = (
    df.where('main_category = "Automotive"')
    .withColumn('secondary_category', 
        F.regexp_replace(F.col('secondary_category'), 'amp;', '')
    )
)

display(df.groupBy('secondary_category').count().orderBy(F.col('count').desc()))

StatementMeta(synsp03, 9, 38, Finished, Available)

SynapseWidget(Synapse.DataFrame, ef3b5529-b57c-41c2-91e9-60a4d2a4ed8c)

In [38]:
df_pandas_automotive = df.pandas_api()
df_pandas_automotive.nunique(axis=0)

StatementMeta(synsp03, 9, 42, Finished, Available)

/opt/spark/python/lib/pyspark.zip/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  [(c, t) for (_, c), t in zip(pdf_slice.iteritems(), arrow_types)]


also_view             203948
brand                  36100
description           372159
feature               291380
rank                  796667
title                 878068
asin                  891307
also_buy              150559
category                5685
main_category              1
image                 337546
price                  34545
secondary_category        15
dtype: int64

In [39]:
df_pandas_automotive.head(20)

StatementMeta(synsp03, 9, 43, Finished, Available)

,asin,also_buy,also_view,brand,category,main_category,description,feature,image,price,rank,title,secondary_category
0,B01A7MLV1W,"[B00VAVSJTC, B00HHGRXA0, B00M07GQO0, B01H06M7Q...",[],Parts Warehouse,"[Automotive, Replacement Parts, Shocks, Strut...",Automotive,14 Piece Kit Control Arms Ball Joints Tie Rod ...,Outer Tie Rod Fit Vehicles W/16 mm Threads Onl...,https://images-na.ssl-images-amazon.com/images...,$15.22,608348,PartsW 14 Piece Kit Control Arms Ball Joints T...,Replacement Parts
1,B001A0B7BK,[],[],TYC,"[Automotive, Lights & Lighting Accessories, Li...",Automotive,05-08 CHEVY UPLANDER MONTANA RIGHT TAIL LIGHT ...,PARTSLINK # GM2801183,,$48.90,1574,05-08 CHEVY UPLANDER MONTANA RIGHT TAIL LIGHT,Lights & Lighting Accessories
2,B00MJV1ZA2,[],[],Cavagna Group,"[Automotive, RV Parts & Accessories, Furnishin...",Automotive,Inlet Size: 1/4 Inch <br>Number Of Stages: 1 <...,Propane Regulator; Without Shutoff Valve; 1/4 ...,https://images-na.ssl-images-amazon.com/images...,,5434851,Cavagna Group 69-C-890-0033B RV Trailer Camper...,RV Parts & Accessories
3,B0014YFB6O,[],"[B01F8E4IX2, B00G1FWOUC, B007HRUSGA, B071HLHYH...",Anco,"[Automotive, Replacement Parts, Windshield Wip...",Automotive,Anco Heavy Duty Specialty Wiper Blades feature...,Features a natural,,$27.03,694399,Anco 5032 Heavy Duty Specialty Wiper - 32&quot;,Replacement Parts
4,B000WJ6LVQ,[],"[B0058PFNT0, B017C0SZL6, B0052XN2ZA, B07DT4ZPJ...",AJ,"[Automotive, Replacement Parts, Exhaust &amp; ...",Automotive,Add style to your vehicle and say good bye to ...,Stainless steel,https://images-na.ssl-images-amazon.com/images...,$43.98,1432943,Stainless steel exhaust tip w/ mirror chrome f...,Replacement Parts
5,B00D6N0CTW,[],[],AutoSmart,"[Automotive, Lights & Lighting Accessories, Bu...",Automotive,These oval shaped LED lighting product functio...,Polycarbonate lens is sonic welded to the housing,https://images-na.ssl-images-amazon.com/images...,$13.99,2020319,SET OF 4 AutoSmart KL-15114C-AE 2&quot; Amber ...,Lights & Lighting Accessories
6,B00KCXR0YO,[],[],Football Stars Ltd.,"[Automotive, Exterior Accessories, Bumper Stic...",Automotive,Show them you are a true fan with this sticker...,Size: 5\ x 4\ (13 x 10 cm),,$3.89,407871,Sheffield FC - England Football Soccer Futbol ...,Exterior Accessories
7,B01APE90W4,[],[],Fusion,"[Automotive, Replacement Parts, Lighting & Ele...",Automotive,FORD Fusion N5F-A08TAA FLIP KEY KEYLESS REMOTE...,2013 - 2016 Ford Fusion,,,13404,FORD Fusion N5F-A08TAA FLIP KEY KEYLESS REMOTE...,Replacement Parts
8,B002656VMS,[],"[B002659W0Q, B00265E1NE, B07DGTFF3K, B003VASHY...",Progressive International,"[Automotive, Lights & Lighting Accessories, Le...",Automotive,Progressive Dynamic PD304,Lights 2Pk Optic Lens Pd304 Progressive Dynamics,,$11.55,141817,Progressive Dynamics PD304 Replacement Frosted...,Lights & Lighting Accessories
9,B00ZC1RCHQ,[],[],ARANA -Key,"[Automotive, Interior Accessories, Keychains]",Automotive,,,https://images-na.ssl-images-amazon.com/images...,,1863138,Sky Blue Smart Keyless Enrty Remote Key Fob Ca...,Interior Accessories
